<a href="https://colab.research.google.com/github/kyanite153/Prog2kakushin/blob/main/work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#大富豪（トランプゲーム）

#ソースコード

In [78]:
import pandas as pd
import random

class CareerPoker:
  def __init__(self, name='player1'):
    self.player_name = name
    self.games = 1
    self.number_of_players = 4
    self.istrump_8 = True
    self.istrump_s3 = True
    self.isreg_sign = True
    self.ismiyakooti = True
    self.data = pd.DataFrame([[name, 0, 0, 0.]], columns=['名前', '1位', '総試合数', '勝率'])
  def is_play_card(self, player, cards=None):
    """
    cardsはset型で受け取る．
    """
    if not cards <= self.state[player]:
      return False
    if 'joker' in cards:
      new_cards = set(cards)
      new_cards.remove('joker')
      for x in self.get_all_card():
        if x == 'joker':
          continue
        new_cards.add(x)
        if self.is_play_card(player, new_cards):
          return True
        new_cards.remove(x)
    print({x[1:] for x in cards})
    if len({x[1:] for x in cards}) == 1:
      #2枚以上の同じ数字のカード
      return True
    #現時点では階段には非対応．
    '''
    a = [self.get_card_strength(x) for x in list(cards)]
    b = [a[x+1]-a[x] for x in range(len(a)-1)]
    if set(b)=={1}:
      return '階段3'
    '''
    return False
  def sort_cards(self, cards):
    l = []
    for x in self.card_strength:
      for card in cards:
        if card[1:] == x:
          l.append(card)
    return l
  def set_game(self):
    while True:
      print('確認/変更したいものを選択．')
      print('1:ラウンド数')
      print('2:8切り')
      print('3:スぺ３返し')
      print('4:記号しばり')
      print('5:都落ち')
      print('6:設定を終了する')
      n = int(input('ここに数字を入力:'))
      if n == 1:
        print(f'現在の設定：{self.games}')
        n = input('ラウンド数：')
        if 1<=n:
          self.games = n
        else:
          print('エラー．ラウンド数は1以上．')
      if n == 1:
        print(f'現在の設定：{self.istrump_8}')
        n = input('8切り (True or False)：')
        if n == 'True':
          self.games = True
        elif n == 'False':
          self.games = False
        else:
          print('エラー．「True」または「False」で指定')
      elif n == 6:
        break
  def get_card_strength(self, card):
    return self.card_strength.index(card[1:])
  def get_all_card(self):
    l = ['3','4','5','6','7','8','9','10','J','Q','K','A','2']
    return {f'h{x}' for x in l}|{f'd{x}' for x in l}|{f's{x}' for x in l}|{f'c{x}' for x in l}|{'joker'}
  def run_game(self):
    com = [CareerPokerCOM(self, f'com{x+1}') for x in range(self.number_of_players-1)]
    players = [self.player_name]+com
    score_data = pd.DataFrame([[players[j]]+[None for i in range(self.games)] for j in range(self.number_of_players)], columns=['name']+[f'{x}試合目' for x in range(1,self.games+1)])
    display(score_data)
    all_card = self.get_all_card()
    self.state = {players[i]:set() for i in range(self.number_of_players)}
    self.state['deck'] = all_card
    p_list = list(players)
    while len(self.state['deck']) != 0:
      if len(p_list) == 0:
        p_list = list(players)
      c = random.choice(list(self.state['deck']))
      p = random.choice(p_list)
      self.state[p].add(c)
      p_list.remove(p)
      self.state['deck'].remove(c)
    self.card_strength = ['3','4','5','6','7','8','9','10','J','Q','K','A','2','oker']
    self.cards_select = set()
    ranking = []
    round = 1
    if round == self.games:
      print(f'ファイナルラウンド')
    else:
      print(f'ラウンド{round}')
    #始めの親を決める
    parent = random.choice(players)
    print(f'最初の親は{parent}です．')
    while True:
      #ゲーム開始
      if parent == self.player_name:
        print(f'{parent}の番です．')
        print(f'場のカード：{list(self.cards_select)}')
        print(f"自分の手札：{self.sort_cards(list(self.state[parent]))}")
        ans = set(input('場に出すカード：').split())
        bool1 = master.is_play_card(parent, ans)
        print(bool1)
      if len(self.state[parent]) == 0:
        #手札が0枚の時
        print(f'{parent}:あがり！')
        ranking.append(parent)
        parent = players[(players.index(parent)+1)%len(players)]
        players.remove(ranking[-1])
      else:
        parent = players[(players.index(parent)+1)%len(players)]
      if len(ranking)+1 == self.number_of_players:
        #ラウンド終了
        ranking.append(parent)
        for player in ranking:
          score_data[f'{round}試合目'][score_data['name']==player] = (len(ranking)-ranking.index(player))*10
        display(score_data)
        #次のラウンドへの準備
        break
    #ゲームセット
    print('ゲーム終了')
    print('結果：不明')

class CareerPokerCOM:
  def __init__(self, master, name='com'):
    self.strength = 1 #1~3
    self.master = master
    self.name = name
  def play_card(self):
    return master.is_play_card(self.name)
  def __repr__(self):
    return self.name

def main():
  global master
  player_name = 'player1'
  while True:
    player_name = input('あなたの名前を入力：')
    if player_name[:3] == 'com':
      print('その名前は使用できません')
    else:
      break
  master=CareerPoker(player_name)
  print('ゲーム開始')
  master.run_game()

if __name__ == '__main__':
  main()


あなたの名前を入力：d
ゲーム開始


,name,1試合目
0,d,None
1,com1,None
2,com2,None
3,com3,None


ファイナルラウンド
最初の親はcom2です．
dの番です．
場のカード：[]
自分の手札：['d5', 'c6', 'd6', 'd8', 'h9', 's10', 'c10', 'd10', 'cJ', 'hJ', 'cK', 'sA', 'c2']
場に出すカード：d5
{'5'}
True
dの番です．
場のカード：[]
自分の手札：['d5', 'c6', 'd6', 'd8', 'h9', 's10', 'c10', 'd10', 'cJ', 'hJ', 'cK', 'sA', 'c2']


KeyboardInterrupt: ignored

In [42]:
master.is_play_card('deck',{'d10','joker'})


{'10', '6'}
{'10', '5'}
{'7', '10'}
{'10'}


True

In [43]:
master.state

{'player1': set(),
 com1: set(),
 com2: set(),
 com3: set(),
 'deck': {'c10',
  'c2',
  'c3',
  'c4',
  'c5',
  'c6',
  'c7',
  'c8',
  'c9',
  'cA',
  'cJ',
  'cK',
  'cQ',
  'd10',
  'd2',
  'd3',
  'd4',
  'd5',
  'd6',
  'd7',
  'd8',
  'd9',
  'dA',
  'dJ',
  'dK',
  'dQ',
  'h10',
  'h2',
  'h3',
  'h4',
  'h5',
  'h6',
  'h7',
  'h8',
  'h9',
  'hA',
  'hJ',
  'hK',
  'hQ',
  'joker',
  's10',
  's2',
  's3',
  's4',
  's5',
  's6',
  's7',
  's8',
  's9',
  'sA',
  'sJ',
  'sK',
  'sQ'}}